# EECS 461 Machine Learning Assignment 2
### 26 Dec. 2017

### Amer Nour Eddin | 213171245

# Path Finding – Is there a path?

Traditionally when we are faced with the task of determining whether a given maze has a path or not we resort to various search or path finding algorithms which are able to answer this question with absolute certainty. In this assignment, however, we will take a different approach to solve this problem by building a classifier that determines (to a certain degree) whether a given maze has a path or not.

+ __training_set_positives.p__ : contains 150 mazes that have a path. Each training example in this file has a label of +1.
+ __training_set_negatives.p__ : contains 150 mazes that do not have a path. Each training example in this file has a label of -1.

The setup for this problem is as follows:

+ Your training set has 300 mazes in total


+ Each training example is a 2 dimensional array where the green squares are represented as zeros and the black squares are represented as ones.
For instance, a given maze could be represented by the following array,
[ [0,0,0,1,0,1,0,0], [0,1,0,0,0,1,0,0], [0,0,1,1,0,0,1,1], ...]


+ Your training set consists of two dictionaries (one for the positive examples and the other for the negative examples) that have these 2 dimensional arrays as their values.

The same logic can be applied to the negative examples as well.

In [1]:
from __future__ import division, print_function, unicode_literals # To support both python 2 and python 3
# import cPickle as pickle # this is for python 2.x
import _pickle as pickle # For python 3.x this is the proper way for calling the cPickle 
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import train_test_split


train_positives = pickle.load(open('training_set_positives.p', 'rb'))
train_negatives = pickle.load(open('training_set_negatives.p', 'rb'))


# train_positives[100]
# train_negatives
# cc = np.array(zz)
# cc



In [2]:

# a0 = cc[0:2, 0:2]
# a1 = cc[0:2, 2:4] 
# a2 = cc[0:2, 4:6]
# a3 = cc[0:2, 6:8]

# b0 = cc[2:4, 0:2]
# b1 = cc[2:4, 2:4]
# b2 = cc[2:4, 4:6]
# b3 = cc[2:4, 6:8]

# c0 = cc[4:6, 0:2]
# c1 = cc[4:6, 2:4]
# c2 = cc[4:6, 4:6]
# c3 = cc[4:6, 6:8]

# d0 = cc[6:8, 0:2]
# d1 = cc[6:8, 2:4]
# d2 = cc[6:8, 4:6]
# d3 = cc[6:8, 6:8]



In [3]:
# print (a0)
# print (a1)
# print (a2)
# print (a3)

# a) Feature Extraction

Your first task is to extract two features from the given dataset.

The first feature you will compute is the proportion of black squares to the total number of squares in the grid. You will write your code for extracting this feature within the function feature1(x).

The second feature you will compute is the sum over all the rows of the maximum number of continuous black squares in each row. You will write your code for extracting this feature within the function feature2(x).

In [4]:
# PART a) Feature Extraction
def feature1(x):
    """This feature computes the proportion of black squares to the
       total number of squares in the grid.
       Parameters
       ----------
       x: 2-dimensional array representing a maze
       Returns
       -------
       feature1_value: type-float
       """
    N = np.count_nonzero(x) # number of black squares or 1's in x
    numrows = len(x) # number of rows in x
    numcols = len(x[0]) # number of columns in x
    total = numrows*numcols # total number of 1's and 0's in x
    feature1_value=N/total # the propotion of 1's in x

    return feature1_value

In [5]:
# print(feature1(train_positives[0]))

In [6]:
def feature2(x):
    """This feature computes the sum of the max of continuous black squares
       in each row
       Parameters
       ----------
       x: 2-dimensional array representing a maze
       Returns
       -------
       feature2_value: type-float
       """
    l = [] # list will contain numbers of continous 1's in each row in x
    
    for i in x:
        count = 0
        result = 0
        for j in i:
            if j == 1:
                count += 1
            else:
                result = max(result, count)
                count= 0
        l.append(max(result, count))
        
    f2 = sum(l)

    feature2_value=f2
    

    return feature2_value

In [7]:
# print(feature2(train_negatives[149]))

# b) Preparing Data

You will use training_set_positives.p and training_set_negatives.p files to create the training data. You should extract features for each grid in these files and put them into an X matrix and also prepare the corresponding label vector y. Keep the same order in training_set_positives.p and training_set_negatives.p (Put training_set_positives.p examples before training_set_negatives.p examples in the X matrix). X and y should be numpy array.

In [8]:
# PART b) Preparing Data
def part_b():
    
    X = []
    y = []
    for i in train_positives.values():
        f1,f2 = feature1(i), feature2(i)
        X.append((f1,f2))
        y.append(1)
    for i in train_negatives.values():
        f1,f2 = feature1(i), feature2(i)
        X.append((f1,f2))
        y.append(0)


    return np.array(X), np.array(y)


In [9]:
# X, y = part_b()
# X.shape
# X
# X.reshape(-1,1)
# X.shape
# np.array(train_negatives[2])

## c) Classification with SGDClassifier

You will built a SGDClassifier model with parameters random_state=0. Train this model with the training data that you created in part b. Write a function that uses your SGDClassifier to predict whether a maze has a path or not and return 1 or 0, respectively.

In [10]:

# PART c) Classification with SGDClassifier

X, y = part_b()
sgd_clf = SGDClassifier(alpha=0.001, n_iter=20, random_state=0)
sgd_clf.fit(X, y)

def part_c(x):
    """
       x: 2-dimensional numpy array representing a maze.
       output: predicted class (1 or 0).
    """
    

    f1, f2 = feature1(x), feature2(x)
    ins = np.array([f1, f2]).reshape(1,-1)

    predicted_class = sgd_clf.predict(ins)
    return predicted_class


In [11]:
asd = np.array(train_positives[149])

PPP = part_c(asd)
# zx = accuracy_score(y, PPP)
# zx*100
PPP

array([1])

## d) Assess the performance of the classifier in part c

Compute precision, recall, and confusion matrix for the classifier in part c on the training set.

In [12]:

# PART d) Assess the performance of the classifier in part c

y_pred = cross_val_predict(sgd_clf, X, y, cv=3)
def part_d():
    
    y_pred = cross_val_predict(sgd_clf, X, y, cv=3)
#     y_pred =  part_c(asd)
    
    precision, recall, confusionmatrix = [precision_score(y, y_pred)], [recall_score(y, y_pred)], [confusion_matrix(y, y_pred)]
    return [precision, recall, confusionmatrix]



In [13]:
# part_d()
# y_pred

[[0.87272727272727268], [0.64000000000000001], [array([[136,  14],
         [ 54,  96]])]]

In [14]:
# y_pred = cross_val_predict(sgd_clf, X, y, cv=3)


# ac = accuracy_score(y, y_pred)
# ac


0.77333333333333332

## e) Classification with RandomForestClassifier

Repeat part c with RandomForestClassifier.

In [15]:

# PART e) Classification with RandomForestClassifier

X, y = part_b()

forest_clf = RandomForestClassifier(random_state=0)
forest_clf.fit(X, y)

def part_e(x):
    """
       x: 2-dimensional numpy array representing a maze.
       output: predicted class (1 or 0).
    """
    f1, f2 = feature1(x), feature2(x)
    ins = np.array([f1, f2]).reshape(1,-1)
    
    predicted_class = forest_clf.predict(ins)
    return predicted_class



In [16]:
# asd = np.array(train_positives[4])

# print(part_e(asd))

## f) Assess the performance of the classifier in part e

Compute precision, recall, and confusion matrix for the classifier in part e on the training set.

In [17]:
# PART f) Assess the performance of the classifier in part e
def part_f():
    
    y_pred = cross_val_predict(forest_clf, X, y, cv=3)
    
    
    precision, recall, confusionmatrix = [precision_score(y, y_pred)], [recall_score(y, y_pred)], [confusion_matrix(y, y_pred)]
    return [precision, recall, confusionmatrix]
    

In [18]:
# part_f()

[[0.84558823529411764], [0.76666666666666672], [array([[129,  21],
         [ 35, 115]])]]

In [19]:
y_pred = cross_val_predict(forest_clf, X, y, cv=3)
# ac = accuracy_score(y, y_pred)
# ac

0.81333333333333335

In [20]:
# cross_val_score(forest_clf, X, y, cv=3, scoring="accuracy")

## g) Your Own Classification Model

Now, prepare your own classifier model with the features of your own, and return the corresponding function as in part c and d. This is the creative part of your assignment where your grade will be determined by how well your classifier works.

In [586]:


# this function will create my features
def features(x):


    a = np.array(x)

    l1 = []  # list will contain numbers of continous 0's in each row in x
    l2 = [] # list will contain numbers of continous 0's in each column in x
    l3 = [] # list will contain numbers of continous 1's in each row in x
    l4 = []# list will contain numbers of continous 1's in each column in x

    for i in a:
        count = 0
        result = 0
        for j in i:
            if j == 0:
                count += 1
            else:
                result = max(result, count)
                count= 0
        l1.append(max(result, count))

    f1 = sum(l1)

    for i in a.T:
        count = 0
        result = 0
        for j in i:
            if j == 0:
                count += 1
            else:
                result = max(result, count)
                count= 0
        l2.append(max(result, count))

    f2 = sum(l2)

    for i in a:
        count = 0
        result = 0
        for j in i:
            if j == 1:
                count += 1
            else:
                result = max(result, count)
                count= 0
        l3.append(max(result, count))

    f3 = sum(l3)

    for i in a.T:
        count = 0
        result = 0
        for j in i:
            if j == 1:
                count += 1
            else:
                result = max(result, count)
                count= 0
        l4.append(max(result, count))

    f4 = sum(l4)
    
    N = np.count_nonzero(x) # number of all 1's
    zeros = 64 - N # number of all 0's
    #f1 is the sum over all the rows of the maximum number of continuous 0's in each row
    #f2 is the sum over all the columns of the maximum number of continuous 0's in each column
    #f3 is the sum over all the rows of the maximum number of continuous 1's in each row
    #f4 is the sum over all the columns of the maximum number of continuous 1's in each column

    return f1+f2/64, N/64




# this function will prepare my X and y
def prepare():
    X = []
    y = []
    for i in train_positives.values():
        f1, f2 = features(i)[0], features(i)[1]
        X.append((f1, f2))
        y.append(1)
    for i in train_negatives.values():
        f1, f2 = features(i)[0], features(i)[1]
        X.append((f1, f2))
        y.append(0)


    return np.array(X), np.array(y)

# X = prepare()[0] #features
# y = prepare()[1] #labels



X, y =  prepare()

# print(len(X))
# print(len(y))

# X_trainP, X_testP, y_trainP, y_testP = train_test_split(X[0:149], y[0:149], test_size=0.2, random_state=0)
# X_trainN, X_testN, y_trainN, y_testN = train_test_split(X[149:300], y[149:300], test_size=0.2, random_state=0)
# print(X_trainP)
# print(len(X_trainP))
# print(y_trainP)
# print(len(y_trainP))
# print(X_testP)
# print(len(X_testP))
# print(y_testP)
# print(len(y_testP))

# print(X_trainN)
# print(len(X_trainN))
# print(y_trainN)
# print(len(y_trainN))
# print(X_testN)
# print(len(X_testN))
# print(y_testN)
# print(len(y_testN))

# # X_train = X_trainP + X_trainN
# # y_train = y_testP + y_trainN

# # X_test = X_testP + X_testN
# # y_test = y_testP + y_testN

# My_Model = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)
# My_Model = RandomForestClassifier(random_state=0)
# My_Model = SGDClassifier(alpha=0.001, n_iter=20, random_state=0)
My_Model = GaussianNB()
# My_Model = tree.DecisionTreeClassifier()
# My_Model = svm.SVC()
# My_Model = svm.LinearSVC()

# from sklearn.model_selection import GridSearchCV

# param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

# knn_clf = KNeighborsClassifier()
# grid_search = GridSearchCV(My_Model, param_grid, cv=5, verbose=3)
# grid_search.fit(X, y)

My_Model.fit(X, y)

# PART g) Your Own Classification Model
def custom_model(x):
    """
       x: 2-dimensional numpy array representing a maze.
       output: predicted class (1 or 0).
    """
#     My_Model.fit(X, y)

    f1, f2 = features(x)[0], features(x)[1]
    ins = np.array([f1, f2]).reshape(1,-1)
    predicted_class = My_Model.predict(ins)
    
    return predicted_class

# this function assesses my model as in function in part d
def asses_my_model():
    
    y_pred = cross_val_predict(My_Model, X, y, cv=3)
   
#     y_pred=My_Model.predict(X)
#     print(y_pred)
#     print(len(y_pred))
    
    
    precision, recall, confusionmatrix = [precision_score(y, y_pred)], [recall_score(y, y_pred)], [confusion_matrix(y, y_pred)]
    return [precision, recall, confusionmatrix]

# prepare()

In [588]:
# for i in range(149):
# print(custom_model(train_negatives[0]))

y_pred = cross_val_predict(My_Model, X, y, cv=3)
# # y_pred=My_Model.predict(X)

# print(custom_model(train_positives[11]))

# accuracy_score(y, y_pred)
# # y_pred

[1]


0.85666666666666669

In [590]:
asses_my_model()

[0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0]
300


[[0.86896551724137927], [0.83999999999999997], [array([[131,  19],
         [ 24, 126]])]]

In [589]:

# cross_val_score(My_Model, X, y, cv=3, scoring="accuracy")